# Neon CRM SDK - Governance Examples

This notebook demonstrates how to use the governance system with the Neon CRM SDK.

Topics covered:
- Simple role-based access (recommended)
- Role with permission overrides
- Using environment variables
- Available roles and their permissions
- Advanced: Switching roles
- Advanced: Permission configuration

In [1]:
import os
from neon_crm import NeonClient
from neon_crm.governance import (
    Role,
    Permission,
    ResourceType,
    PermissionConfig
)
from neon_crm.governance.access_control import PermissionError

print("✅ Imports successful")

✅ Imports successful


## Available Roles

The SDK provides several built-in roles for common use cases:

In [2]:
print("Available Roles:")
print("=" * 50)

roles_info = {
    "viewer": "Read-only access to most resources",
    "editor": "Read and write access to most resources",
    "admin": "Full access to all resources",
    "fundraiser": "Focused on donation and fundraising activities",
    "event_manager": "Focused on event management",
    "volunteer_coordinator": "Focused on volunteer management"
}

for role, description in roles_info.items():
    print(f"  • {role}: {description}")

Available Roles:
  • viewer: Read-only access to most resources
  • editor: Read and write access to most resources
  • admin: Full access to all resources
  • fundraiser: Focused on donation and fundraising activities
  • event_manager: Focused on event management
  • volunteer_coordinator: Focused on volunteer management


## 1. Simple Role-Based Usage (Recommended)

The simplest way to use governance is to specify a role when creating the client.

In [3]:
# Create a client with a fundraiser role# Governance is enabled by defaultclient = NeonClient(    org_id=os.getenv("NEON_ORG_ID"),    api_key=os.getenv("NEON_API_KEY"),    default_role="fundraiser"  # Can also use Role.FUNDRAISER)try:    # This will work - fundraisers can read accounts    accounts = list(client.accounts.search({        "searchFields": [{"field": "Account Type", "operator": "NOT_BLANK"}],        "outputFields": ["Account ID", "First Name", "Last Name"],        "pagination": {"currentPage": 0, "pageSize": 5}    }))    print(f"✓ Successfully retrieved {len(accounts)} accounts")        # Fundraisers can also create donations (commented out for safety)    # donation_data = {...}    # client.donations.create(donation_data)    print("✓ Fundraiser role has permission to create donations")    except PermissionError as e:    print(f"✗ Permission denied: {e}")

## 2. Role with Permission Overrides

You can start with a base role and override permissions for specific resources.

In [4]:
# Start with viewer role (read-only) but allow creating donationsclient_with_overrides = NeonClient(    org_id=os.getenv("NEON_ORG_ID"),    api_key=os.getenv("NEON_API_KEY"),    default_role="viewer",    permission_overrides={        "donations": {"read", "write"},  # Can use strings        ResourceType.CAMPAIGNS: {Permission.READ, Permission.WRITE}  # Or enums    })try:    # This will work - viewers can read    accounts = list(client_with_overrides.accounts.search({        "searchFields": [{"field": "Account Type", "operator": "NOT_BLANK"}],        "outputFields": ["Account ID", "First Name"],        "pagination": {"currentPage": 0, "pageSize": 2}    }))    print(f"✓ Read {len(accounts)} accounts (viewer permission)")        # This will work - we overrode to allow donation writes    print("✓ Can create donations (permission override)")        # This will fail - viewers can't create accounts (no override)    # client_with_overrides.accounts.create({...})  # Would raise PermissionError    print("✗ Cannot create accounts (viewer restriction, no override)")    except PermissionError as e:    print(f"✗ Permission denied: {e}")

## 3. Using Environment Variables

You can configure the default role using the `NEON_DEFAULT_ROLE` environment variable.

In [5]:
# Set role via environment variable (or use .env file)
# os.environ["NEON_DEFAULT_ROLE"] = "editor"

client_env = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY")
    # Role loaded from NEON_DEFAULT_ROLE environment variable
    # Defaults to "viewer" if not set
)

print(f"Governance enabled: {client_env.governance_enabled}")
if client_env.user_permissions:
    print(f"Role: {client_env.user_permissions.role}")
else:
    print("Using default 'viewer' role")

Governance enabled: True
Role: viewer


## 4. Advanced: Switching Roles

For different operations, you can create separate clients with different roles.

In [6]:
# Create an admin client for privileged operationsadmin_client = NeonClient(    org_id=os.getenv("NEON_ORG_ID"),    api_key=os.getenv("NEON_API_KEY"),    default_role="admin")try:    # Admin can do anything    accounts = list(admin_client.accounts.search({        "searchFields": [{"field": "Account Type", "operator": "NOT_BLANK"}],        "outputFields": ["Account ID"],        "pagination": {"currentPage": 0, "pageSize": 2}    }))    print(f"✓ Admin accessed {len(accounts)} accounts")except PermissionError as e:    print(f"✗ Admin permission denied: {e}")# Create a viewer client for read-only operationsviewer_client = NeonClient(    org_id=os.getenv("NEON_ORG_ID"),    api_key=os.getenv("NEON_API_KEY"),    default_role="viewer")try:    # Viewer can read    accounts = list(viewer_client.accounts.search({        "searchFields": [{"field": "Account Type", "operator": "NOT_BLANK"}],        "outputFields": ["Account ID"],        "pagination": {"currentPage": 0, "pageSize": 2}    }))    print(f"✓ Viewer accessed {len(accounts)} accounts")        # Viewer cannot create (commented out)    # viewer_client.accounts.create({...})  # Would raise PermissionError    print("✗ Viewer cannot create accounts (by design)")    except PermissionError as e:    print(f"✗ Viewer permission denied: {e}")

## 5. Advanced: Permission Configuration

For multi-user scenarios, you can create a permission configuration with multiple user profiles.

In [7]:
# Create a permission configuration
config = PermissionConfig()

# Add users to the configuration
config.add_user("fundraiser1", Role.FUNDRAISER)
config.add_user("admin1", Role.ADMIN)
config.add_user("event_manager1", Role.EVENT_MANAGER)

print("✓ Created permission configuration with 3 users")

# Create a client with the configuration
client_config = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY"),
    permission_config=config
)

# Set user by ID (looks up in configuration)
if client_config.set_user_by_id("fundraiser1"):
    print("✓ Successfully set user permissions from config")
    
    # Get current user's permissions for donations
    perms = config.get_effective_permissions("fundraiser1", ResourceType.DONATIONS)
    print(f"  Fundraiser1 donation permissions: {[p.value for p in perms]}")
else:
    print("✗ User not found in configuration")

# List all users with donation write permission
users_with_write = config.list_users_with_permission(ResourceType.DONATIONS, Permission.WRITE)
print(f"  Users who can create donations: {users_with_write}")

✓ Created permission configuration with 3 users
✓ Successfully set user permissions from config
  Fundraiser1 donation permissions: ['read', 'update', 'write']
  Users who can create donations: ['fundraiser1', 'admin1']


## Summary

### Key Takeaways

1. **Simple Role-Based** (Recommended): Just pass `default_role` parameter
   ```python
   client = NeonClient(default_role="fundraiser")
   ```

2. **Permission Overrides**: Customize specific resource permissions
   ```python
   client = NeonClient(
       default_role="viewer",
       permission_overrides={"donations": {"read", "write"}}
   )
   ```

3. **Environment Variables**: Use `NEON_DEFAULT_ROLE` for configuration

4. **Multiple Roles**: Create separate clients for different permission levels

5. **Advanced Config**: Use `PermissionConfig` for multi-user scenarios

### Best Practices

- ✅ Use the simplest approach that meets your needs (usually `default_role`)
- ✅ Use `viewer` role by default for read-only operations
- ✅ Use specific roles (`fundraiser`, `event_manager`) for focused access
- ✅ Only use `admin` role when truly necessary
- ✅ Test permission restrictions before deploying to production

### Available Roles

- `viewer` - Read-only access
- `editor` - Read/write for most resources
- `fundraiser` - Donations, accounts, campaigns, pledges
- `event_manager` - Events, registrations, attendees
- `volunteer_coordinator` - Volunteers, activities
- `admin` - Full access to everything

For more details, see the [Permissions Documentation](../docs/permissions.md).